In [2]:
!pip install mercury
!pip install sympy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.3/728.3 kB 33.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 10.7 MB

In [3]:
import sympy as sp
import numpy as np
import scipy.optimize as opt
from scipy import integrate
import matplotlib.pyplot as plt
import re
import mercury as mr
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

# Define the text input for the user's question
question_input = mr.Text(
    label="Enter your mathematical question:",
    value="What is the derivative of sin(x)?"
)

# Helper function to determine the operation
def determine_operation(question):
    question_lower = question.lower()
    if any(keyword in question_lower for keyword in ["solve", "equation", "="]):
        return "Solve Expression"
    elif any(keyword in question_lower for keyword in ["plot", "graph", "visualize", "draw"]):
        return "Visualize Function"
    elif "integrate numerically" in question_lower:
        return "Integrate Numerically"
    elif "definite integral" in question_lower or "evaluate integral from" in question_lower:
        return "Definite Integral"
    elif "integrate" in question_lower or "antiderivative" in question_lower:
        return "Integrate Symbolically"
    elif any(keyword in question_lower for keyword in ["solve linear system", "system of equations"]):
        return "Solve Linear System"
    elif any(keyword in question_lower for keyword in ["differentiate", "derivative"]):
        return "Differentiate"
    else:
        return "Unknown"

# Helper functions to extract information
def extract_expression(question):
    match = re.search(r'solve.*?:?\s*(.*)', question, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        # Try to find an equation in the question
        match = re.search(r'(.+)=.+', question)
        if match:
            return question
    return question  # Default to returning the whole question

def extract_function(question):
    match = re.search(r'(?:function|f\(x\)|y=)\s*(.*)', question, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        # Try to extract from "plot ..." or "graph ..."
        match = re.search(r'(?:plot|graph|visualize)\s*(.*)', question, re.IGNORECASE)
        if match:
            return match.group(1)
    return None

def extract_function_to_differentiate(question):
    # Try to extract the function to differentiate
    match = re.search(r'(?:derivative of|differentiate|what is the derivative of)\s*(.*?)[\?\.]?$', question, re.IGNORECASE)
    if match:
        function_str = match.group(1).strip()
        # Remove trailing punctuation
        function_str = function_str.rstrip('.?')
        return function_str
    return None

def extract_range(question):
    # Extract numerical values for range
    matches = re.findall(r'from\s*([\d\.-]+)\s*to\s*([\d\.-]+)', question)
    if matches:
        start, end = matches[0]
        return float(start), float(end)
    else:
        # Default range
        return 0.0, 6.28

def extract_limits(question):
    matches = re.findall(r'from\s*([\d\.-]+)\s*to\s*([\d\.-]+)', question)
    if matches:
        lower, upper = matches[0]
        return float(lower), float(upper)
    else:
        # Default limits
        return 0.0, 1.0

# Helper function to safely parse expressions
def safe_parse(expression):
    try:
        allowed_symbols = {'x'}
        allowed_functions = {
            'sin': sp.sin, 'cos': sp.cos, 'tan': sp.tan, 'exp': sp.exp,
            'log': sp.log, 'sqrt': sp.sqrt, 'Abs': sp.Abs,
            'asin': sp.asin, 'acos': sp.acos, 'atan': sp.atan,
            'sinh': sp.sinh, 'cosh': sp.cosh, 'tanh': sp.tanh,
            'pi': sp.pi, 'E': sp.E, 'ln': sp.ln
        }

        # Split the expression if it contains an '=' for equations
        if '=' in expression:
            lhs, rhs = expression.split('=')
            expr = sp.Eq(sp.sympify(lhs.strip(), locals=allowed_functions),
                         sp.sympify(rhs.strip(), locals=allowed_functions))
        else:
            expr = sp.sympify(expression, locals=allowed_functions)

        symbols_in_expr = expr.free_symbols
        if not all(str(sym) in allowed_symbols for sym in symbols_in_expr):
            raise ValueError("Invalid symbols in expression.")
        return expr
    except Exception as e:
        logging.error(f"Error parsing expression '{expression}': {e}")
        raise ValueError(f"Invalid expression: {e}")

# Main logic
operation_value = determine_operation(question_input.value)

if operation_value == "Unknown":
    mr.Markdown("**Error:** Unable to determine the operation from the question.")
else:
    if operation_value == "Solve Expression":
        try:
            expression_str = extract_expression(question_input.value)
            x = sp.symbols('x')
            expr = safe_parse(expression_str)
            solutions = sp.solve(expr, x)
            steps = sp.pretty(sp.solve(expr, x, dict=True))
            result = str(solutions)
            mr.Markdown(f"**Result:** {result}")
            mr.Markdown(f"**Steps:**\n```\n{steps}\n```")
        except Exception as e:
            logging.error(f"Error in solving expression: {e}")
            mr.Markdown(f"**Error:** {e}")
    elif operation_value == "Visualize Function":
        try:
            function_str = extract_function(question_input.value)
            if not function_str:
                mr.Markdown("**Error:** Could not extract function to visualize.")
                raise ValueError("Function extraction failed.")
            x = sp.symbols('x')
            expr = safe_parse(function_str)
            func = sp.lambdify(x, expr, 'numpy')
            range_start, range_end = extract_range(question_input.value)
            x_vals = np.linspace(range_start, range_end, 400)
            y_vals = func(x_vals)

            plt.figure()
            plt.plot(x_vals, y_vals)
            plt.xlabel('x')
            plt.ylabel(f'f(x) = {function_str}')
            plt.title('Function Visualization')
            plt.grid(True)

            mr.Pyplot(plt.gcf())
            plt.close()
        except Exception as e:
            logging.error(f"Error in visualizing function: {e}")
            mr.Markdown(f"**Error:** {e}")
    elif operation_value == "Integrate Symbolically":
        try:
            expression_str = extract_expression(question_input.value)
            x = sp.symbols('x')
            expr = safe_parse(expression_str)
            integral = sp.integrate(expr, x)
            steps = f"∫ {expression_str} dx = {integral}"
            result = str(integral)
            mr.Markdown(f"**Result:** {result}")
            mr.Markdown(f"**Steps:**\n```\n{steps}\n```")
        except Exception as e:
            logging.error(f"Error in symbolic integration: {e}")
            mr.Markdown(f"**Error:** {e}")
    elif operation_value == "Integrate Numerically":
        try:
            expression_str = extract_expression(question_input.value)
            x = sp.symbols('x')
            expr = safe_parse(expression_str)
            func = sp.lambdify(x, expr, 'numpy')
            lower_limit, upper_limit = extract_limits(question_input.value)
            result, error = integrate.quad(func, lower_limit, upper_limit)
            mr.Markdown(f"**Integral:** {result}")
            mr.Markdown(f"**Error Estimate:** {error}")
        except Exception as e:
            logging.error(f"Error in numerical integration: {e}")
            mr.Markdown(f"**Error:** {e}")
    elif operation_value == "Definite Integral":
        try:
            expression_str = extract_expression(question_input.value)
            lower_limit, upper_limit = extract_limits(question_input.value)
            x = sp.symbols('x')
            expr = safe_parse(expression_str)
            integral_result = sp.integrate(expr, (x, lower_limit, upper_limit))
            indefinite_integral = sp.integrate(expr, x)
            steps = f"∫ {expression_str} dx = {indefinite_integral}"
            result = str(integral_result)
            mr.Markdown(f"**Definite Integral from {lower_limit} to {upper_limit}:** {result}")
            mr.Markdown(f"**Steps:**\n```\n{steps}\n```")
        except Exception as e:
            logging.error(f"Error in definite integration: {e}")
            mr.Markdown(f"**Error:** {e}")
    elif operation_value == "Differentiate":
        try:
            function_str = extract_function_to_differentiate(question_input.value)
            if not function_str:
                mr.Markdown("**Error:** Could not extract function to differentiate.")
                raise ValueError("Function extraction failed.")
            x = sp.symbols('x')
            expr = safe_parse(function_str)
            derivative = sp.diff(expr, x)
            steps = f"d/dx {function_str} = {derivative}"
            result = str(derivative)
            mr.Markdown(f"**Result:** {result}")
            mr.Markdown(f"**Steps:**\n```\n{steps}\n```")
        except Exception as e:
            logging.error(f"Error in differentiation: {e}")
            mr.Markdown(f"**Error:** {e}")
    elif operation_value == "Solve Linear System":
        mr.Markdown("**Error:** Solving linear systems from natural language input is not implemented.")
        # Implement parsing for linear systems if needed

mercury.Text

**Result:** cos(x)

**Steps:**
```
d/dx sin(x) = cos(x)
```